Общая ячейка для всех импортов

In [78]:
import requests
from tqdm import tqdm
from bs4 import BeautifulSoup
from fake_useragent import UserAgent
from nltk import word_tokenize
from collections import Counter
from pymorphy2 import MorphAnalyzer
morph = MorphAnalyzer()
from sklearn.metrics import accuracy_score

ua = UserAgent(verify_ssl=False)
session = requests.session()

Сегодня мы будем смотреть на отзывы клиентов банков и по их лексике пытаться предсказать их опыт общения с банком!
Возможно, возникнут проблемы: профессионально-экономическую лексику сложновато по тональности оценить, а ее явно будет много
Пройдемся по сайту, соберем все отзывы с оценкой "1" или "5" (да, этот сайт это кошмар веб-разработчика в плане наименований классов); добавим ссылки на нужные отзывы в списки

In [108]:
neg_reviews = []
positive_reviews = []
page_number = 0
for i in tqdm(range(100)):
    page_number += 1
    url = f'https://www.sravni.ru/banki/otzyvy/?page={page_number}'
    req = session.get(url, headers={'User-Agent': ua.random})
    req.encoding = 'utf-8'
    page = req.text
    soup = BeautifulSoup(page, 'html.parser')
    reviews = soup.find_all('div', {'class': '_3qkdy _7QkVd'})
    for review in reviews:
        rating = review.find('span', {'class': '_1OBr6'})
        link = review.find('a', {'class': 'mrfZC'}).attrs['href']
        if rating:
            for child in rating.children:
                rating = child
            if rating == '1':
                neg_reviews.append(link)
            elif rating == '5':
                positive_reviews.append(link)
    

100%|████████████████████████████████████████████████████████████████████████████████| 100/100 [02:23<00:00,  1.44s/it]


Проверим, сколько у нас получилось отзывов

In [109]:
print ('amount of positives:', len (positive_reviews))
print ('amount of negatives:', len (neg_reviews))

amount of positives: 825
amount of negatives: 345


Это маленькая песочница, чтобы посмотреть, будет ли работать большая функция, гуляющая по ссылкам и вытаскивающая тексты: посмотрим заодно на один пример

In [114]:
link = neg_reviews[10]
new_link = f'https://sravni.ru{link}'
print(new_link)
req = session.get(new_link, headers={'User-Agent': ua.random})
req.encoding = 'utf-8'
req = req.text
req = BeautifulSoup(req, 'html.parser')
new_text = req.find('div', {'class': "_3lM0q"}).text.lower()
if new_text:
    print(new_text)
else:
    print('damn')

https://sravni.ru/bank/gazprombank/otzyv/379671/?page=3
здравствуйте всем. ужасное обслуживание. 25 09 2020 года я решил взять кредит , узнал про ставку 6.9 процента и пришел в это отделение банка. сорок минут я сидел дожидаясь очереди на прием к оператору ну бог с ним финансы есть финансы. через 40 минут загорелся мой номер на табло и я попал к оператору зайцевой анне. вот тут я понял почему очередь идет так медленно. одну минуту она занималась моим вопросом потом ей принесла какие-то бумаги соседний оператор и на этом закончилось мое обслуживание. зайцева анна в течении 10 минут 3 раза поговорила по сотовому телефону , набирая какую-то информацию на компьютере, и задавая мне не чего не значащие вопросы. после десяти минут такого обслуживания я возмутился и забрал документы собрался уходить тут она меня поразила полностью своей фразой : "да успокойтесь вы садитесь я сейчас вами займусь". обалдеть так эти 10 минут она занималась не мной , а чем? может быть она проверяла домашнюю работу

Выглядит правдоподобно. Сделаем это функцией, распарсим и посчитаем статистику!

In [115]:
def get_full_reviews(link_list):
    words = []
    for link in link_list:
        new_link = f'https://sravni.ru{link}'
        req = session.get(new_link, headers={'User-Agent': ua.random})
        req.encoding = 'utf-8'
        req = req.text
        req = BeautifulSoup(req, 'html.parser')
        new_text = req.find('div', {'class': '_3lM0q'})
        if new_text:
            new_text = new_text.text.lower()
        new_text = word_tokenize(new_text)
        for word in new_text:
            if word.isalpha():
                words.append(morph.parse(word)[0][2])
    words = dict(Counter(words))
    return words

Возьмем из общего списка ту часть, на которой будем учиться - по 300 штук, чтобы сбалансировать разное количество отзывов в двух блоках - нам надо, чтобы обоим на тестовый датасет хватило

In [116]:
positive_dict = get_full_reviews(positive_reviews[:300])
neg_dict = get_full_reviews(neg_reviews[:300])

Посмотрим, сколько у нас получилось слов

In [117]:
print("there are ", len(positive_dict), " words in the positive dictionary")
print("there are", len (neg_dict), " words in the negative dictionary")

there are  2982  words in the positive dictionary
there are 4847  words in the negative dictionary


Хорошо, теперь можно заняться множествами. Отсеем все малоупотребимые слова:

In [118]:
def make_set(dictionary):
    word_set = []
    for word, freq in dictionary.items():
        if freq > 2:
            word_set.append(word)
    word_set = set(word_set)
    return word_set

Применим эту полезную функцию для наших позитивного и негативного тренировочных датасетов:
(сначала делим на тест и тренировку, потом обрабатываем - можно получить утечку данных в обучении!)

In [119]:
positive_set = make_set(positive_dict) 
neg_set = make_set(neg_dict)

Вычеркнем все, что нам не нравится: избавимся от шумов и общих слов

In [120]:
full_set = positive_set & neg_set
positive_set = positive_set.difference(full_set)
neg_set = neg_set.difference(full_set)

И вот она, функция для тональностей! Сообщаем ей ссылки на тестовый датасет и просим оценить тональность отзывов

In [122]:
def tone(link):
    positives = negatives = 0
    words = get_full_reviews([link,])
    for word, freq in words.items():
        if word in positive_set:
            positives += 1*freq
        elif word in neg_set:
            negatives += 1*freq
    if positives > negatives:
        result = 'positive'
    elif positives < negatives:
        result = 'negative'
    else:
        result = 'fifty-fifty'
    return result

Венец творения - последняя функция, поможет нам определить точность; пусть подскажет нам разницу между предсказанием и реальностью

In [123]:
def main_acc(positives, negatives):
    Y_hat = []
    Y = []
    for text in positives:
        Y_hat.append(tone(text))
        Y.append('positive')
    for text in negatives:
        Y_hat.append(tone(text))
        Y.append('negative')
    return accuracy_score(Y_hat, Y)

In [125]:
print(main_acc(positive_reviews[300:345], neg_reviews[300:]))

0.7555555555555555


Что можно улучшить? Вероятно, сделать что-то с аббревиатурами (вычеркнуть все слова, написанные только капсом?), порисовать красивых графиков, отсортировать отзывы по разным банкам. Возможно, для некоторых банков вероятность того, что отзыв о нем отрицательный:)